## Summary results
For each of the four experiments (binary predictor/outcome; prescription count/binary outcome; binary predictor/age onset; prescrition count/age onset):
1. Range of sample sizes
2. Range of % of each sex (can just show female)
3. Range of mean age
4. Range of odds ratios 
5. % statistically significant after correction

In [1]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm

def importdata(ehr, predictor, outcome):
    """Import data from the EHR and return a pandas dataframe.

    Parameters
    ----------
    ehr : str
        'sinai' or 'ukb'
    predictor : str
        'binary' or 'prescription'
    outcome : str
        'binary' or 'age_onset'

    Returns
    -------
    df : pandas.DataFrame
        Dataframe with the predictor and outcome variables.
    """
    if ehr == 'sinai':
        file_ehr = 'MSDW1794_V3'
    elif ehr == 'ukb':
        file_ehr = 'ukbiobank'
    else:
        raise ValueError('Invalid EHR: %s' % ehr)

    if predictor == 'binary':
        file_pred = 'binary_exposure'
    elif predictor == 'prescription':
        file_pred = 'prescription_count'
    elif predictor == 'aud':
        file_pred = 'aud'
    else:
        raise ValueError('Invalid predictor: %s' % predictor)

    if outcome == 'binary':
        file_out = 'binary_outcome'
    elif outcome == 'age_onset':
        file_out = 'age_onset_ncd'
    else:
        raise ValueError('Invalid outcome: %s' % outcome)

    if file_pred == 'aud':
        if ehr=='sinai': path = f"{file_ehr}/voe_outputs/aud/controlsNoAUDDX/{file_out}/controlVarOUD/analyses/period_summaries/"
        elif ehr=='ukb': path = f'{file_ehr}/voe_outputs/aud/controlsNoAUDDX/{file_out}/controlVarSUD/analyses/period_summaries/'
    else:
        path = f"{file_ehr}/voe_outputs/opioids/controlsLessThan3Opioids/{file_pred}/{file_out}/controlVarOUD/analyses/period_summaries/"
    datasets = []

    for enrollment_year in range(1989,2020):
        if os.path.exists(path + f'voe_{enrollment_year}_{enrollment_year+3}.csv'):
            ds = pd.read_csv(path + f'voe_{enrollment_year}_{enrollment_year+3}.csv')
            datasets.append(ds)
    allexpts = pd.concat(datasets)
    # print(allexpts.shape)

    #check for duplicate experiments
    allexpts = allexpts.drop_duplicates()
    # print(allexpts.shape)

    #remove years with low sample sizes
    # print(allexpts.shape)
    if ehr == 'sinai': allexpts = allexpts[allexpts.start_enroll>=2008]
    elif ehr == 'ukb': allexpts = allexpts[(allexpts.start_enroll>=2004) & (allexpts.start_enroll<=2010)]
    if predictor != 'aud': allexpts = allexpts[allexpts['hx_MAT']==0] 

    # print(allexpts.shape)

    # create total N column
    allexpts['total_N'] = allexpts['control_N'] + allexpts['opioid_N']

    #add corrected p-values, OR confidence intervals, percentage of each group with NCD
    if 'age_onset_ncd' not in path:
        allexpts['coef'] = np.exp(allexpts['coef'])
        allexpts['.025'] = np.exp(allexpts['.025'])
        allexpts['.975'] = np.exp(allexpts['.975'])

    # correct for multiple comparisons
    allexpts['bonferroni'] = sm.stats.multipletests(allexpts['p'], alpha=0.05, method='bonferroni')[1]
    allexpts['bh_p'] = sm.stats.multipletests(allexpts['p'], alpha=0.05, method='fdr_bh')[1]

    # percentage of NCD for each group
    allexpts['opi_percent_ncd'] = 100 * (allexpts['num_opioid_ncd'] / allexpts['opioid_N'])
    allexpts['con_percent_ncd'] = 100 * (allexpts['num_control_ncd'] / allexpts['control_N'])

    # percentage of sex for total sample
    allexpts['total_female%'] = ((allexpts['control_female%'] * allexpts['control_N']) + (allexpts['opioid_female%'] * allexpts['opioid_N'])) / allexpts['total_N']

    # mean age for total sample
    allexpts['total_mean_age'] = ((allexpts['control_AgeMean'] * allexpts['control_N']) + (allexpts['opioid_AgeMean'] * allexpts['opioid_N'])) / allexpts['total_N']

    df = allexpts.copy()
    return df

def get_summary(predictor, outcome):
    sinai = importdata('sinai', predictor, outcome)
    ukb = importdata('ukb', predictor, outcome)
    df = pd.concat([sinai, ukb])
    # print(predictor, outcome, '\n',
    # 'N', min(df.total_N), max(df.total_N), '\n',
    # 'Opioid N', min(df.opioid_N), max(df.opioid_N), '\n',
    # 'Control N', min(df.control_N), max(df.control_N), '\n',
    # 'Female %', min(df['total_female%']), max(df['total_female%']), '\n',
    # 'Mean age', min(df['total_mean_age']), max(df['total_mean_age']), '\n',
    # 'Coef', f'Min: {min(df.coef)},', f'Max: {max(df.coef)},', f'Median: {np.median(df.coef)}', '\n',
    # '% significant', sum(df.bh_p<0.05)/len(df), f'Median p: {np.median(df.bh_p)}','\n\n')
    
    df = sinai.copy()
    rcoef, rp = summ_stats(df.coef, -np.log10(df.bh_p))
    print('SINAI', '\n',
          'N', min(df.total_N), max(df.total_N), '\n',
    'Opioid N', min(df.opioid_N), max(df.opioid_N), '\n',
    'Control N', min(df.control_N), max(df.control_N), '\n',
    'Opioid NCD%', min(df.opi_percent_ncd), max(df.opi_percent_ncd), '\n',
    'Control NCD%', min(df.con_percent_ncd), max(df.con_percent_ncd), '\n',
    'Female %', min(df['total_female%']), max(df['total_female%']), '\n',
    'Mean age', min(df['total_mean_age']), max(df['total_mean_age']), '\n',
    'Coef', f'Min: {min(df.coef)},', f'Max: {max(df.coef)},', f'Median: {np.median(df.coef)}', '\n',
    '% significant', sum(df.bh_p<0.05)/len(df), f'Median p: {np.median(df.bh_p)}','\n',
    'RES:', rcoef, '\n',
    'RP:', rp, '\n\n')

    df = ukb.copy()
    rcoef, rp = summ_stats(df.coef, -np.log10(df.bh_p))
    print('UKB', '\n',
            'N', min(df.total_N), max(df.total_N), '\n',
    'Opioid N', min(df.opioid_N), max(df.opioid_N), '\n',
    'Control N', min(df.control_N), max(df.control_N), '\n',
    'Opioid NCD%', min(df.opi_percent_ncd), max(df.opi_percent_ncd), '\n',
    'Control NCD%', min(df.con_percent_ncd), max(df.con_percent_ncd), '\n',
    'Female %', min(df['total_female%']), max(df['total_female%']), '\n',
    'Mean age', min(df['total_mean_age']), max(df['total_mean_age']), '\n',
    'Coef', f'Min: {min(df.coef)},', f'Max: {max(df.coef)},', f'Median: {np.median(df.coef)}', '\n',
    '% significant', sum(df.bh_p<0.05)/len(df), f'Median p: {np.median(df.bh_p)}','\n',
    'RES:', rcoef, '\n',
    'RP:', rp, '\n\n')

    
    return df

def update_figures(predictor, outcome):
    df = get_summary(predictor, outcome)
    
def summ_stats(coefs, pvals):
    '''Summary statistics:
    including the 1st, 50th (median), and 99th percentile of effect size and p-values
    the “relative odds ratio” (Rcoef) as the ratio of the 99th and 1st percentile odds ratio
    the “relative p-value” (RP) as the difference between the 99th and 1st percentile of -log10(adjusted p-value). '''

    #calculate relative effect size
    coefs = sorted(abs(coefs))
    rcoef = round(coefs[int(len(coefs)*.99)] / coefs[int(len(coefs)*.01)],2)
    
    #calculate relative p-value    
    ps = sorted(pvals)
    rp = round(ps[int(len(ps)*.99)] - ps[int(len(ps)*.01)],2)

    return rcoef, rp

In [3]:
get_summary('prescription', 'binary')

In [21]:
get_summary('ukb', 'prescription', 'binary')

(12016, 28)
(12016, 28)
(12016, 28)
(2112, 28)
ukb prescription binary 
 N 16599 186540 
 Female % 0.500210830672851 0.5703712579915274 
 Mean age 58.098693792099034 69.6107443931923 
 Coef 0.9974383569534021 1.0047918534092297 



In [5]:
sinai_bin_binary = importdata('sinai', 'binary', 'binary')
sinai_rx_binary = importdata('sinai', 'prescription', 'binary')
sinai_bin_age = importdata('sinai', 'binary', 'age_onset')
sinai_rx_age = importdata('sinai', 'prescription', 'age_onset')

ukb_bin_binary = importdata('ukb', 'binary', 'binary')
ukb_rx_binary = importdata('ukb', 'prescription', 'binary')
ukb_bin_age = importdata('ukb', 'binary', 'age_onset')
ukb_rx_age = importdata('ukb', 'prescription', 'age_onset')

In [6]:
print(sinai_bin_binary[sinai_bin_binary.bh_p<0.05].shape[0] / sinai_bin_binary.shape[0], 
ukb_bin_binary[ukb_bin_binary.bh_p<0.05].shape[0] / ukb_bin_binary.shape[0])

0.9405864197530864 0.8390151515151515


In [11]:
sinai_bin_binary.columns

Index(['control_N', 'opioid_N', 'control_AgeMean', 'control_AgeSD',
       'opioid_AgeMean', 'opioid_AgeSD', 'control_male%', 'control_female%',
       'opioid_male%', 'opioid_female%', 'coef', 'stderr', '.025', '.975', 'p',
       'num_control_ncd', 'num_opioid_ncd', 'followup_time', 'start_enroll',
       'end_enroll', 'opioid_rx_enroll', 'ncd_age_threshold', 'hx_sickle',
       'hx_hiv', 'hx_aud', 'hx_tobacco', 'hx_sud_covar', 'hx_MAT', 'total_N',
       'bonferroni', 'bh_p', 'opi_percent_ncd', 'con_percent_ncd',
       'total_female%', 'total_mean_age'],
      dtype='object')

In [12]:
sinai_bin_binary.loc[:,['bh_p', 'p']]

,bh_p,p
0,1.585663e-08,7.677496e-09
2,8.228837e-08,4.254106e-08
4,3.454060e-07,1.973558e-07
6,1.060986e-06,6.594320e-07
8,2.168458e-07,1.205536e-07
...,...,...
566,7.634795e-04,6.771757e-04
568,8.253990e-05,6.833743e-05
570,4.555796e-04,4.003898e-04
572,2.555044e-04,2.188348e-04


In [22]:
sinai_bin_binary.columns

Index(['control_N', 'opioid_N', 'control_AgeMean', 'control_AgeSD',
       'opioid_AgeMean', 'opioid_AgeSD', 'control_male%', 'control_female%',
       'opioid_male%', 'opioid_female%', 'coef', 'stderr', '.025', '.975', 'p',
       'num_control_ncd', 'num_opioid_ncd', 'followup_time', 'start_enroll',
       'end_enroll', 'opioid_rx_enroll', 'ncd_age_threshold', 'hx_sickle',
       'hx_hiv', 'hx_aud', 'hx_tobacco', 'hx_sud_covar', 'hx_MAT', 'total_N',
       'bonferroni', 'bh_p', 'opi_percent_ncd', 'con_percent_ncd',
       'total_female%', 'total_mean_age'],
      dtype='object')

In [35]:
smoking_percent = {'control':[], 'opioid':[]}
aud_percent = {'control':[], 'opioid':[]}
hiv_percent = {'control':[], 'opioid':[]}
sickle_percent = {'control':[], 'opioid':[]}

for pop in os.listdir('./ukbiobank/voe_outputs/opioids/controlsLessThan3Opioids/binary_exposure/binary_outcome/controlVarOUD/populations/'):
    if pop != '.DS_Store':
        pop_df = pd.read_csv('./ukbiobank/voe_outputs/opioids/controlsLessThan3Opioids/binary_exposure/binary_outcome/controlVarOUD/populations/' + pop)
        for label,name in zip([0,1], ['control', 'opioid']):
            df = pop_df[pop_df['label']==label]
            smoking_percent[name].append(sum(df['tobacco']) / len(df))
            aud_percent[name].append(sum(df['aud']) / len(df))
            hiv_percent[name].append(sum(df['hiv']) / len(df))
            sickle_percent[name].append(sum(df['sickle']) / len(df))

In [36]:
#UKB
 
print(
    'Smoking: ', min(smoking_percent['control']), max(smoking_percent['control']), min(smoking_percent['opioid']), max(smoking_percent['opioid']), '\n',
    'AUD: ', min(aud_percent['control']), max(aud_percent['control']), min(aud_percent['opioid']), max(aud_percent['opioid']), '\n',
    'HIV: ', min(hiv_percent['control']), max(hiv_percent['control']), min(hiv_percent['opioid']), max(hiv_percent['opioid']), '\n',
    'Sickle: ', min(sickle_percent['control']), max(sickle_percent['control']), min(sickle_percent['opioid']), max(sickle_percent['opioid']), '\n',
)


Smoking:  0.015234975931715548 0.06768241497177156 0.03636363636363636 0.2222222222222222 
 AUD:  0.0013288055514543038 0.01009530184403856 0.0 0.037037037037037035 
 HIV:  0.0006609385327164573 0.005496989743711777 0.0 0.03636363636363636 
 Sickle:  0.0 0.000577311652073511 0.0 0.0022002200220022 



In [34]:
#MSDW
 
print(
    'Smoking: ', min(smoking_percent['control']), max(smoking_percent['control']), min(smoking_percent['opioid']), max(smoking_percent['opioid']), '\n',
    'AUD: ', min(aud_percent['control']), max(aud_percent['control']), min(aud_percent['opioid']), max(aud_percent['opioid']), '\n',
    'HIV: ', min(hiv_percent['control']), max(hiv_percent['control']), min(hiv_percent['opioid']), max(hiv_percent['opioid']), '\n',
    'Sickle: ', min(sickle_percent['control']), max(sickle_percent['control']), min(sickle_percent['opioid']), max(sickle_percent['opioid']), '\n',
)


Smoking:  0.009754162051349876 0.03136592457691571 0.07081377151799687 0.292910447761194 
 AUD:  0.0034718203911584308 0.010577753863505844 0.01471836965751486 0.055970149253731345 
 HIV:  0.002893183659298692 0.022827837453288927 0.018361581920903956 0.19962686567164178 
 Sickle:  0.00032271944922547335 0.000778650993620753 0.001564945226917058 0.009328358208955223 



In [30]:
pop_df.head()

,Unnamed: 0,MRN,AGE,DOB,RACE,SEX,YOB,age,label,ncd,sickle,hiv,aud,tobacco,sud,opioid_count,MAT
0,0,1003267,80.0,1942-04-20,WHITE,Female,1942,69,0,0,0,0,0,0,0,0.0,0
1,1,1004882,76.0,1946-01-23,WHITE,Male,1946,65,0,0,0,0,0,0,0,0.0,0
2,2,1005516,99.0,1923-06-01,WHITE,Female,1923,88,0,1,0,0,0,0,0,1.0,0
3,3,1007351,72.0,1950-08-03,WHITE,Female,1950,61,0,0,0,0,0,0,0,1.0,0
4,4,1008956,74.0,1948-06-30,WHITE,Female,1948,63,0,0,0,0,0,0,0,0.0,0
